<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/HW5/4106HW5P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F
import torchtext

In [ ]:
SOS_token = 0
EOS_token = 1

In [ ]:
engtofra = [
    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")
]


In [ ]:
class Translate(Dataset):
  def __init__(self, engtofra, word2idx):
    self.engtofra = engtofra
    self.word2idx = word2idx


  def __len__(self):
    return len(self.engtofra)

  def __getitem__(self, idx):
    inputsen, targetsen = self.engtofra[idx]
    inputtokens = [self.word2idx[word] for word in inputsen.split(' ')] + [EOS_token]
    targettokens = [self.word2idx[word] for word in targetsen.split(' ')] + [EOS_token]
    return torch.tensor(inputtokens), torch.tensor(targettokens)

In [ ]:
word2idx = {"SOS": SOS_token, "EOS": EOS_token}
for pair in engtofra:
  for word in pair[0].split()  + pair[1].split():
    if word not in word2idx:
      word2idx[word] = len(word2idx)

In [ ]:
translation_dataset = Translate(engtofra, word2idx)
dataloader = DataLoader(translation_dataset, batch_size=1, shuffle=True)

In [ ]:
idx2word = {idx: word for word, idx in word2idx.items()}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


In [ ]:
class Trans(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, layers, nhead, dropout = .1):
    super(Trans, self).__init__()
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.embeddingtarget = nn.Embedding(output_size, hidden_size)
    self.transformer = nn.Transformer(hidden_size, nhead, layers, dim_feedforward=hidden_size * 4, dropout = .1)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, src, target):
      srcembedded = self.embedding(src)
      targetembedded = self.embeddingtarget(target)
      srcembedded = srcembedded.permute(1, 0, 2)
      targetembedded = targetembedded.permute(1, 0, 2)
      output = self.transformer(srcembedded, targetembedded)
      output = self.fc(output)
      return self.softmax(output)

  def generate_padding_mask(self, sequence):
        mask = (sequence == word2idx["EOS"])
        return mask

  def generate_subsequent_mask(self, sequence):
        mask = (torch.triu(torch.ones(sequence.size(1), sequence.size(1))) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [ ]:
input_size = len(word2idx)
output_size = len(word2idx)
hidden_size = 128
layers = 3
nhead = 2
lr = .005
epochs = 100

In [ ]:
model = Trans(len(word2idx), hidden_size, len(word2idx), layers, nhead).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def train_loop(model, dataloader, optimizer, criterion, epochs):
  for epoch in range(epochs):
    losses = 0
    corrects = 0
    exs = 0
    model.train()
    for src, target in dataloader:
      src = src.to(device)
      target = target.to(device)
      optimizer.zero_grad()
      output = model(src, target[:, :-1])
      outputdim = output.shape[-1]
      output = output[:, 1:].contiguous().view(-1, outputdim)
      target = target[:, 1:].contiguous().view(-1)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      losses += loss.item()
      corrects += (output.argmax(1) == target).sum().item()
      exs += output.shape[0]

    lossvg = losses / len(dataloader)
    accuracy = corrects/exs
    print(f"Epoch {epoch+1}: Loss {lossvg:.4f}, Accuracy {accuracy:.4f}")

In [ ]:
model = Trans(input_size, output_size, hidden_size, layers, nhead).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

epochs = 10
train_loop(model, dataloader, optimizer, criterion, epochs)